In [5]:
from catboost import CatBoostRegressor, Pool, cv
from sklearn.model_selection import cross_val_score
import copy
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline, make_union
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.metrics import roc_auc_score
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

In [6]:
from sklearn.neighbors import BallTree
from IPython.display import clear_output

In [7]:
PATH_PROJECT = '..'
PATH_DATA = f'{PATH_PROJECT}/data'

In [12]:
df = pd.read_pickle(f'{PATH_DATA}/processed/df.pickle')
df_voronoi_features = pd.read_pickle(f'{PATH_DATA}/processed/voronoi_features.pickle')
df = df.merge(df_voronoi_features, on='id')

In [13]:
for iso in [5,10,15,30]:
    df[f'ratio_people2area_iso_{iso}'] = df.eval(f'people_voronoi_iso_{iso} / area_voronoi_iso_{iso}')

In [14]:
for iso in [5,10,15,30]:
    df[f'sales2people_iso_{iso}'] = df.eval(f'sales_per_month * ratio_people2area_iso_{iso}')

In [15]:
iso = 10

mask_train = df['split'] == 'train'

ball = BallTree(df.loc[mask_train, ["lat_rad", "lon_rad"]].values, metric='haversine')
dic_index2id = dict(enumerate(df.loc[mask_train, 'id'].values))

k = 20

distances, indices = ball.query(df[["lat_rad", "lon_rad"]].values, k = k)
top_n = pd.DataFrame(indices, index=df['id'])
top_n = top_n \
    .applymap(dic_index2id.get) \
    .stack() \
    .rename_axis(['id', 'rank']) \
    .to_frame('id_2') \
    .reset_index()
top_n['dist'] = distances.reshape(-1) * 6371000
top_n = top_n.query('id != id_2')
top_n['rank'] = top_n.groupby('id')['dist'].rank(method='first').astype(int)
top_n['sales'] = top_n['id_2'].map(df.set_index('id')[f'sales2people_iso_{iso}'])
top_n[f'sales_iso_{iso}'] = top_n.groupby('id')['sales'].cumsum() / top_n['rank']
top_n = top_n.query(f'rank != {k}').set_index(['id', 'rank'])[[f'sales_iso_{iso}']].unstack()
top_n.columns = [f'{c1}_top_{c2}' for c1, c2 in top_n.columns]
top_n = top_n.reset_index()

In [16]:
top_n.head()

,id,sales_iso_10_top_1,sales_iso_10_top_2,sales_iso_10_top_3,sales_iso_10_top_4,sales_iso_10_top_5,sales_iso_10_top_6,sales_iso_10_top_7,sales_iso_10_top_8,sales_iso_10_top_9,sales_iso_10_top_10,sales_iso_10_top_11,sales_iso_10_top_12,sales_iso_10_top_13,sales_iso_10_top_14,sales_iso_10_top_15,sales_iso_10_top_16,sales_iso_10_top_17,sales_iso_10_top_18,sales_iso_10_top_19
0,1,30.856423,30.997265,31.739370,33.887957,32.569492,30.347176,30.482181,29.828607,30.461739,28.961237,28.134773,28.247059,27.652192,28.295303,28.407839,27.373650,26.339637,25.874254,26.030670
1,2,24.092227,26.516884,30.948366,32.913422,34.004133,31.143463,27.817868,29.644621,28.921023,28.487159,27.480773,26.421490,25.225371,24.888703,25.512257,24.013568,24.794399,24.392248,25.798013
2,3,0.033731,2.024341,2.884893,2.463893,2.606143,2.379141,5.414636,7.033370,11.679488,13.482410,17.317698,22.685788,21.603859,26.985865,27.863982,26.621825,30.862225,33.065638,32.804309
3,4,41.544938,44.889525,43.279747,33.022929,27.350240,22.995290,20.626386,18.208116,30.067392,27.271360,25.966552,25.860682,31.834857,33.491958,35.634545,36.668822,37.281600,37.874174,41.616017
4,5,59.703124,55.443485,61.976680,62.457015,60.429420,54.835063,60.642739,60.994123,60.402768,58.697183,57.662028,62.779387,62.249599,61.209197,60.834368,61.630032,61.061813,59.678893,59.054553


In [288]:
new_df.shape

(26017, 63)

In [17]:
mask_train = df['split'] == 'train'

df_train = df[mask_train][['region_new', 'sales_per_month']]
sales_per_month = df_train.groupby(['region_new'])['sales_per_month'].transform('sum') - df_train['sales_per_month']
cnt = df_train.groupby(['region_new'])['sales_per_month'].transform('count') - 1
df_train['region_sales'] = sales_per_month / cnt
region_sales = df_train.groupby('region_new')['region_sales'].mean().to_dict()
df['region_sales'] = df['region_new'].map(region_sales)

In [18]:
mask_train = df['split'] == 'train'

df_train = df[mask_train][['locality_new', 'sales_per_month']]
sales_per_month = df_train.groupby(['locality_new'])['sales_per_month'].transform('sum') - df_train['sales_per_month']
cnt = df_train.groupby(['locality_new'])['sales_per_month'].transform('count') - 1
df_train['locality_sales'] = sales_per_month / cnt
locality_sales = df_train.groupby('locality_new')['locality_sales'].mean().to_dict()
df['locality_sales'] = df['locality_new'].map(locality_sales)

In [19]:
mask_train = df['split'] == 'train'

df_train = df[mask_train][['country_code', 'sales_per_month']]
sales_per_month = df_train.groupby(['country_code'])['sales_per_month'].transform('sum') - df_train['sales_per_month']
cnt = df_train.groupby(['country_code'])['sales_per_month'].transform('count') - 1
df_train['country_sales'] = sales_per_month / cnt
country_sales = df_train.groupby('country_code')['country_sales'].mean().to_dict()
df['country_sales'] = df['country_code'].map(country_sales)

In [21]:
df[['sales_per_month', 'region_sales', 'locality_sales', 'country_sales']].corr()['sales_per_month']

sales_per_month    1.000000
region_sales       0.244117
locality_sales     0.269246
country_sales      0.092992
Name: sales_per_month, dtype: float64

In [24]:
clms = ['id', 'region_sales', 'locality_sales', 'ratio_people2area_iso_5', 'ratio_people2area_iso_10',
       'ratio_people2area_iso_15', 'ratio_people2area_iso_30',]

stats = df[clms].merge(top_n, on='id')

In [26]:
stats.head()

,id,region_sales,locality_sales,ratio_people2area_iso_5,ratio_people2area_iso_10,ratio_people2area_iso_15,ratio_people2area_iso_30,sales_iso_10_top_1,sales_iso_10_top_2,sales_iso_10_top_3,...,sales_iso_10_top_10,sales_iso_10_top_11,sales_iso_10_top_12,sales_iso_10_top_13,sales_iso_10_top_14,sales_iso_10_top_15,sales_iso_10_top_16,sales_iso_10_top_17,sales_iso_10_top_18,sales_iso_10_top_19
0,12017,2588.5,2601.372881,0.058278,0.058183,0.040564,0.027858,144.911617,107.156774,136.464590,...,88.563205,89.357258,83.438420,84.564536,81.663602,79.938096,77.942335,77.725141,77.337480,81.545127
1,752,2588.5,2601.372881,0.009505,0.008139,0.005503,0.003406,18.561176,12.036192,19.360028,...,26.609166,30.902242,28.941762,32.024883,32.916855,35.156067,36.522561,35.662279,34.117561,32.799053
2,23380,2588.5,2402.296296,0.012623,0.011807,0.008427,0.007111,13.296015,20.910543,39.160073,...,27.921498,29.203731,30.106182,31.601216,31.005444,30.079582,32.513504,31.185870,31.895069,31.151962
3,11961,2588.5,2601.372881,0.028824,0.009840,0.012578,0.017529,102.686097,90.178471,78.895468,...,112.980653,107.290171,106.457473,102.883671,96.182403,92.018722,91.500415,90.485687,89.314367,86.200655
4,18307,2588.5,2402.296296,0.026184,0.026706,0.026386,0.012757,28.525071,20.910543,28.089021,...,28.017053,26.184354,27.338419,29.046358,28.633075,27.865372,26.745270,26.244804,28.621333,28.050527


In [27]:
stats.to_pickle(f'{PATH_DATA}/processed/stats.pickle')